In [1]:
import os
import numpy as np
import pandas as pd
from scipy.misc import imread
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
import fuzzy_classification as fc

In [2]:
print(tf.__version__)

1.1.0


In [3]:
traindata_df = fc.traindata_df
target_df = fc.target_df
# To stop potential randomness
seed = 50
rng = np.random.RandomState(seed)
random_seed = 25
train_x = traindata_df.values
target_y = target_df.values

#split_size = int(train_x.shape[0] * 0.7)
train_x, val_x, target_y, val_y = train_test_split(
        train_x, target_y, test_size=0.33, random_state=random_seed)

In [4]:
def dense_to_one_hot(labels_dense, num_classes=11):
    """Convert class labels from scalars to one-hot vectors"""
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset.astype(int) + labels_dense.ravel().astype(int)] = 1
    
    return labels_one_hot

In [5]:
def preproc(unclean_batch_x):
    """Convert values to range 0-1"""
    temp_batch = unclean_batch_x
    
    return temp_batch

In [6]:
def batch_creator(batch_size, dataset_length, step_name):
    """Create batch with random samples and return appropriate format"""
    batch_mask = rng.choice(dataset_length, batch_size)
    batch_x = train_x[[batch_mask]].reshape(-1, 28)
    batch_x = preproc(batch_x)
    if step_name == 'train':
        batch_y = target_y[batch_mask]
        batch_y = dense_to_one_hot(batch_y)
        
    return batch_x, batch_y

In [7]:
batch_size = 50
batch_x, batch_y = batch_creator(batch_size, train_x.shape[0], 'train')

In [8]:
# number of neurons in each layer
input_num_units = 28
n_units_hl1 = 500
n_units_hl2 = 500
n_units_hl3 = 500
n_units_hl4 = 500

n_classes = 11
output_num_units = 11
learning_rate = 0.01

# define placeholders
x = tf.placeholder('float', [None, input_num_units])
y = tf.placeholder('float', [None, output_num_units])
keep_prob = tf.placeholder('float')
# set remaining variables
epochs = 7
batch_size = 50

### define weights and biases of the neural network

weights = {
    'hidden_1' : tf.Variable(tf.random_normal([input_num_units, n_units_hl1], seed=seed)),
    'hidden_2' : tf.Variable(tf.random_normal([n_units_hl1, n_units_hl2], seed=seed)),
    'hidden_3' : tf.Variable(tf.random_normal([n_units_hl2, n_units_hl3], seed=seed)),
    'hidden_4' : tf.Variable(tf.random_normal([n_units_hl3, n_units_hl4], seed=seed)),
    'output' : tf.Variable(tf.random_normal([n_units_hl4, output_num_units], seed=seed))
}

biases = {
    'hidden_1' : tf.Variable(tf.random_normal([n_units_hl1], seed=seed)),
    'hidden_2' : tf.Variable(tf.random_normal([n_units_hl2], seed=seed)),
    'hidden_3' : tf.Variable(tf.random_normal([n_units_hl3], seed=seed)),
    'hidden_4' : tf.Variable(tf.random_normal([n_units_hl4], seed=seed)),
    'output' : tf.Variable(tf.random_normal([output_num_units], seed=seed))
}

In [9]:
### create neural network computational graph
def feed_forword():
    hidden_layer_1 = tf.add(tf.matmul(x, weights['hidden_1']), biases['hidden_1'])
    hidden_layer_1 = tf.nn.relu(hidden_layer_1)

    hidden_layer_2 = tf.add(tf.matmul(hidden_layer_1, weights['hidden_2']), biases['hidden_2'])
    hidden_layer_2 = tf.nn.relu(hidden_layer_2)
    hidden_layer_2 = tf.nn.dropout(hidden_layer_2, keep_prob)

    hidden_layer_3 = tf.add(tf.matmul(hidden_layer_2, weights['hidden_3']), biases['hidden_3'])
    hidden_layer_3 = tf.nn.relu(hidden_layer_3)
    hidden_layer_3 = tf.nn.dropout(hidden_layer_3, keep_prob)

    hidden_layer_4 = tf.add(tf.matmul(hidden_layer_3, weights['hidden_4']), biases['hidden_4'])
    hidden_layer_4 = tf.nn.relu(hidden_layer_4)
    hidden_layer_4 = tf.nn.dropout(hidden_layer_4, keep_prob)

    output_layer = tf.matmul(hidden_layer_4, weights['output']) + biases['output']

    return output_layer

In [24]:
with tf.Session() as sess:
    output_layer = feed_forword()
    def train_model(train_x, target_y):
        last_epoch_cost = np.array([])
        acc = np.array([])
        #output_layer = feed_forword()
        for k in range(0, 5):
            train_x, val_x, target_y, val_y = train_test_split(
                train_x, target_y, test_size=0.33, random_state=random_seed)
            cost = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                        logits=output_layer, labels=y)))
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
            init = tf.global_variables_initializer()
            sess.run(init)
            for epoch in range(epochs):
                avg_cost = 0
                total_batch = int(traindata_df.shape[0] / batch_size)
                for i in range(total_batch):
                    batch_x, batch_y = batch_creator(batch_size, train_x.shape[0], 'train')
                    _, c = sess.run([optimizer, cost], feed_dict = {x: batch_x, y: batch_y, keep_prob: 0.75})

                    avg_cost += c / total_batch
            last_epoch_cost = np.append(last_epoch_cost, avg_cost)
            pred_temp = tf.equal(tf.argmax(output_layer, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(pred_temp, 'float'))
            feed_dict = {x: val_x.reshape(-1, 28), y: dense_to_one_hot(val_y), keep_prob: 1}

            val_acc = sess.run(accuracy,feed_dict)
            acc = np.append(acc, val_acc)
            
        print ('last_epoch_cost :')
        print (last_epoch_cost)
        print ('accuracy :')
        print (acc)
        print ('Average score : ', np.average(acc))
    
    def prediction(test_x):
        predict = tf.argmax(output_layer, 1)
        feed_dict = {x: test_x.reshape(-1, 28), keep_prob : 1}
        classification = sess.run(predict, feed_dict)
        print ('Classification :')
        print (classification)
train_model(train_x, target_y)
#prediction()

last_epoch_cost :
[ 1533.69170908   596.92365676   635.40781344   160.57314359    59.94749685]
accuracy :
[ 0.98675495  0.99009901  1.          0.9333334   0.96774191]
Average score :  0.975585854053


In [32]:
#################################
#####Conncet to AWS database#########
import MySQLdb

db = MySQLdb.connect(host='phpmyadmin.c2bpuzfzdybb.us-west-2.rds.amazonaws.com', 
                      port=3306, user='phpMyAdmin', passwd='phpMyAdmin', db='clouddatasensors')

if (db):
    # Carry out normal procedure
    print ('Connection successful')
else:
    # Terminate
    print ("Connection unsuccessful")
cursor = db.cursor()

Connection successful


In [33]:
#sql = 'SELECT Temperature, Humidity, Rain, Vel, Wt FROM sensor ORDER BY did DESC LIMIT 1'
sql = 'SELECT output FROM Road_Weight ORDER BY id DESC LIMIT 1'
cursor.execute(sql)
results = cursor.fetchall()
print (results)

((2,),)


In [25]:
test_x = fc.predict_fuzz_mem(40, 0, 0, 20, 10, 0, 0, 0, 50)
print (test_x)
print (test_x.reshape(-1, 28))
prediction(test_x)

[ 0.          0.          1.          0.          0.          0.          1.
  0.          1.          0.          0.          0.          0.          1.
  1.          0.          0.          0.          1.          0.          1.
  0.          0.          1.          0.          0.          0.
  0.88235294]
[[ 0.          0.          1.          0.          0.          0.          1.
   0.          1.          0.          0.          0.          0.          1.
   1.          0.          0.          0.          1.          0.          1.
   0.          0.          1.          0.          0.          0.
   0.88235294]]
Classification :
[7]
